# Extract Playlist into DataFrame

In [1]:
import pandas as pd

In [2]:
playlist_file = "Resources/Top50_Playlist_by_Country.xlsm"

In [3]:
playlist_df = pd.read_excel(playlist_file)
playlist_df

,Continents,Country,Spotify URL,Spotify Playlist ID
0,North America,USA,https://open.spotify.com/playlist/37i9dQZEVXbL...,37i9dQZEVXbLRQDuF5jeBp
1,NaN,CAN,https://open.spotify.com/playlist/37i9dQZEVXbK...,37i9dQZEVXbKj23U1GF4IR
2,South America,BRA,https://open.spotify.com/playlist/37i9dQZEVXbM...,37i9dQZEVXbMXbN3EUUhlg
3,NaN,COL,https://open.spotify.com/playlist/37i9dQZEVXbO...,37i9dQZEVXbOa2lmxNORXQ
4,NaN,ARG,https://open.spotify.com/playlist/37i9dQZEVXbM...,37i9dQZEVXbMMy2roB9myp


# Extract Songs into DataFrame

In [4]:
###Search Audio API EXAMPLE###

# search_url_audio = "https://api.spotify.com/v1/audio-features/"
# audio_url = search_url_audio + "59PYgzOiOjGDzjDT5N5oOX"

# audio_json = requests.get(audio_url, headers = token_headers).json()

import os
import time
import requests
import numpy as np
import pandas as pd
from requests.auth import HTTPBasicAuth
from client import client_id, client_secret

## Create directory (folder), if exists overwrite files inside directory
path = "CSV Files"

try:  
    os.mkdir(path)

except OSError:  
    print ("Creation of the directory %s failed" % path)
    
else:  
    print ("Successfully created the directory %s " % path)

## Top 50 Playlist by Country Excel
top50_playlist_df = pd.read_excel("Resources/Top50_Playlist_by_Country.xlsm")

## This request grants us a temporary token to use to access the json data from spotify.
token_url = "https://accounts.spotify.com/api/token"
headers = {'content-type': 'application/x-www-form-urlencoded'}

token_params = {"grant_type" : "client_credentials"}

token_json = requests.post(token_url, params = token_params, headers = headers, auth = (client_id, client_secret)).json()
token = token_json['access_token']

# Required header to access spotify api
token_headers = {'authorization': f"Bearer {token}"}

# Counter is used to grab country name from top50_playlist_df
counter = 0

# Search api endpoint for playlist and audio features
search_url_playlist = "https://api.spotify.com/v1/playlists/"
search_url_audio = "https://api.spotify.com/v1/audio-features/"

print("--------------------------\nBegin Exporting CSV Files\n--------------------------")

# Loops through each playlist ID from top50_playlist_df
for playlist_id in top50_playlist_df["Spotify Playlist ID"]:
    
    playlist_url = search_url_playlist + playlist_id
    
    # Store entire playlist json data into variable
    playlist_json = requests.get(playlist_url, headers = token_headers).json()
    
    # List to store json data
    ID = []
    artist_name = []
    artist_id = []
    track_name = []
    track_id = []
    release_date = []
    popularity = []
    danceability = []
    energy = []
    
    # Number of tracks within each playlist (should be 50 songs)
    number_of_tracks = len(playlist_json["tracks"]["items"])
    
    # Loops through each playlist in playlist_json
    for i in range(number_of_tracks):
        time.sleep(0.01)
        
        try:
            # Store track json as variable for faster access
            playlist_tracks = playlist_json["tracks"]["items"][i]["track"]
            
            # Append artist name, artist ID, track name, track ID, release date, and popularity
            artist_name.append(playlist_tracks["album"]["artists"][0]["name"])
            artist_id.append(playlist_tracks["album"]["artists"][0]["id"])
            track_name.append(playlist_tracks["name"])
            
            # Store track ID as variable for faster access
            track_ids = playlist_tracks["id"]
            track_id.append(track_ids)
            
            release_date.append(playlist_tracks["album"]["release_date"])
            popularity.append(playlist_tracks["popularity"])
            
            # Accessing audio endpoint from spotify
            audio_url = search_url_audio + track_ids

            audio_json = requests.get(audio_url, headers = token_headers).json()
            
            # Append danceability and energy score
            danceability.append(audio_json["danceability"])
            energy.append(audio_json["energy"])
            
            # Append playlist ID
            ID.append(counter)
            
        except (KeyError, IndexError):
            danceability.append("NaN")
            energy.append("NaN")
            
            print(f"Could not find audio ID | {track_ids}")
    
    #Create DataFrame to store into excel
    artist_country_df = pd.DataFrame({"ID" : ID,
                                      "Artists Name" : artist_name,
                                      "Artist ID" : artist_id,
                                      "Track Name" : track_name,
                                      "Track ID" : track_id,
                                      "Release Date" : release_date,
                                      "Popularity" : popularity,
                                      "Danceability" : danceability,
                                      "Energy" : energy})
    
    # Country Name
    country = top50_playlist_df["Country"][counter]
    
    # Export dataframe of each country into seperate excel files
    artist_country_df.to_csv(f"{path}/{country}_top_50.csv")
    
    counter += 1
    
print("--------------------------\nDone Exporting CSV Files\n--------------------------")

Creation of the directory CSV Files failed
--------------------------
Begin Exporting CSV Files
--------------------------
--------------------------
Done Exporting CSV Files
--------------------------


In [5]:
print("-------------------------------------\nImporting Excel Files into DataFrame\n-------------------------------------")

from pathlib import Path

# Root directory to Top Country CSV folder
rootdir = Path('CSV Files')

# Loops through each csv file in Top Country CSV folder
file_list = [f for f in rootdir.glob("*.csv") if f.is_file()]

# Store country name
country_name = str(file_list[0]).split("\\")[-1].split("_")[0]

# Convert first csv into DataFrame
song_df = pd.read_csv(file_list[0])

# Loop starts at second file and ends at last
for file in file_list[1:]:
    country_name = str(file).split("\\")[-1].split("_")[0]
    
    # Convert excel into DataFrame
    df = pd.read_csv(file)
    
    # Add each country DataFrame onto the first
    song_df = song_df.append(df)

print("-------------------------------------\nImporting into DataFrame Complete\n-------------------------------------")

-------------------------------------
Importing Excel Files into DataFrame
-------------------------------------
-------------------------------------
Importing into DataFrame Complete
-------------------------------------


In [6]:
song_df.head()

,Unnamed: 0,ID,Artists Name,Artist ID,Track Name,Track ID,Release Date,Popularity,Danceability,Energy
0,0,4,Daddy Yankee,4VMYDCV2IEDYJArk749S6m,Con Calma,5w9c2J52mkdntKOmRLeM2m,2019-01-24,100,0.737,0.860
1,1,4,Dalex,0KPX4Ucy9dk82uj4GpKesn,Pa Mí - Remix,224EVTFiLsdQignUFLvDeH,2019-02-07,93,0.688,0.620
2,2,4,Neo Pistea,01m2XZ7m7rAz6KY3scTdaV,"Tumbando el Club (feat. C.R.O., Obiewanshot, Y...",0OWcpVA83HI8Pb5DybDuoX,2019-03-27,76,0.882,0.777
3,3,4,Anuel Aa,2R21vXR83lH98kGeO99Y66,Secreto,5W83ErFkO3aKAIS1WMi6u0,2019-01-15,96,0.807,0.803
4,4,4,Pedro Capó,4QVBYiagIaa6ZGSPMbybpy,Calma - Remix,5iwz1NiezX7WWjnCgY5TH4,2018-10-05,96,0.826,0.773


# Extract Country ISO into DataFrame

In [7]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import bs4
import pandas as pd

In [8]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [9]:
url = "https://www.nationsonline.org/oneworld/country_code_list.htm"

In [10]:
browser.visit(url)
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

In [11]:
html_data = soup.find_all('tr',class_="border1")

In [12]:
result = []
for i in html_data:
    result.append(i.text)

In [13]:
dict_= {"Country":[],
           "Alpha_2":[],
           "Alpha_3_Code":[],
           "UN_Code":[]}

for i in result:
    split_list = i.split("\n")
    if len(split_list)>6:
        dict_["Country"].append(split_list[2])
        dict_["Alpha_2"].append(split_list[3])
        dict_["Alpha_3_Code"].append(split_list[4])
        dict_["UN_Code"].append(split_list[5])

In [14]:
countryISO_df = pd.DataFrame(dict_)

In [15]:
countryISO_df.head()

,Country,Alpha_2,Alpha_3_Code,UN_Code
0,Aland Islands,AX,ALA,248
1,Albania,AL,ALB,008
2,Algeria,DZ,DZA,012
3,American Samoa,AS,ASM,016
4,Andorra,AD,AND,020


# Transform Playlist DataFrame

In [16]:
# Create a filtered dataframe from specific columns
playlist_col = ["Country", "Spotify URL", "Spotify Playlist ID"]

In [23]:
playlist_transformed = playlist_df[playlist_col].copy()

# Rename the column headers
playlist_transformed = playlist_transformed.rename(columns={"Country": "country_code",
                                                          "Spotify URL": "spotify_url",
                                                          "Spotify Playlist ID": "playlist_id"})

playlist_transformed.index.names = ["id"]

In [24]:
playlist_transformed

,country_code,spotify_url,playlist_id
id,,,
0,USA,https://open.spotify.com/playlist/37i9dQZEVXbL...,37i9dQZEVXbLRQDuF5jeBp
1,CAN,https://open.spotify.com/playlist/37i9dQZEVXbK...,37i9dQZEVXbKj23U1GF4IR
2,BRA,https://open.spotify.com/playlist/37i9dQZEVXbM...,37i9dQZEVXbMXbN3EUUhlg
3,COL,https://open.spotify.com/playlist/37i9dQZEVXbO...,37i9dQZEVXbOa2lmxNORXQ
4,ARG,https://open.spotify.com/playlist/37i9dQZEVXbM...,37i9dQZEVXbMMy2roB9myp


# Transform Song DataFrame

In [22]:
# Create a filtered dataframe from specific columns
song_col = ["ID", "Track Name", "Track ID", "Artist ID"]

song_transformed = song_df[song_col].copy()

# Rename the column headers
song_transformed = song_transformed.rename(columns={"ID": "id",
                                                        "Track Name": "song_name",
                                                        "Track ID": "song_id",
                                                        "Artist ID": "artist_id"})

song_transformed.set_index("id").head()

,song_name,song_id,artist_id
id,,,
4,Con Calma,5w9c2J52mkdntKOmRLeM2m,4VMYDCV2IEDYJArk749S6m
4,Pa Mí - Remix,224EVTFiLsdQignUFLvDeH,0KPX4Ucy9dk82uj4GpKesn
4,"Tumbando el Club (feat. C.R.O., Obiewanshot, Y...",0OWcpVA83HI8Pb5DybDuoX,01m2XZ7m7rAz6KY3scTdaV
4,Secreto,5W83ErFkO3aKAIS1WMi6u0,2R21vXR83lH98kGeO99Y66
4,Calma - Remix,5iwz1NiezX7WWjnCgY5TH4,4QVBYiagIaa6ZGSPMbybpy


# Transform Country DataFrame

In [26]:
# Create a filtered dataframe from specific columns
country_transformed = countryISO_df.copy()

# Rename the column headers
country_transformed = country_transformed.rename(columns={"Country": "country_name",
                                                        "Alpha_2": "alpha_2",
                                                        "Alpha_3_Code": "country_code",
                                                        "UN_Code": "un_code"})

country_transformed.set_index("country_code")

,country_name,alpha_2,un_code
country_code,,,
ALA,Aland Islands,AX,248
ALB,Albania,AL,008
DZA,Algeria,DZ,012
ASM,American Samoa,AS,016
AND,Andorra,AD,020
AGO,Angola,AO,024
AIA,Anguilla,AI,660
ATA,Antarctica,AQ,010
ATG,Antigua and Barbuda,AG,028
